# With TF

In [3]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [15]:
train_generator= ImageDataGenerator(
    rotation_range= 15, 
    width_shift_range= 0.1, 
    height_shift_range= 0.1,
    horizontal_flip= True,
    vertical_flip= True
)
test_generator= ImageDataGenerator()

train_path= 'dataset/dogs-vs-cat/train'
valid_path= 'dataset/dogs-vs-cat/valid'
test_path= 'dataset/dogs-vs-cat/test'

train_batches= train_generator.flow_from_directory(
    train_path,
    target_size= (224, 224)
)
val_batches= train_generator.flow_from_directory(
    valid_path,
    target_size= (224, 224)
)
test_batches= test_generator.flow_from_directory(
    test_path,
    target_size= (224, 224)
)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [16]:
train_batches.target_size

(224, 224)

In [17]:
train_batches.batch_size

32

In [23]:
base_model= vgg16.VGG16(include_top= False, 
                        weights= 'imagenet', 
                        input_shape= (224, 224, 3),
                        )
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
# freeze all layers
for layer in base_model.layers:
    layer.trainable= False

In [25]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [31]:
last_layer= base_model.get_layer(index= -1)
last_layer_output= last_layer.output
last_layer_output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [37]:
x= Flatten()(last_layer_output)
x= Dense(64, activation= 'relu', name= 'FC_2')(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(2, activation= 'softmax', name= 'softmax')(x)

new_model= Model(inputs= base_model.input, outputs= x)
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [38]:
new_model.compile(loss= 'binary_crossentropy', 
                  optimizer= Adam(learning_rate= 0.0001),
                  metrics= ['accuracy'])

In [40]:
epochs= 10
new_model.fit(train_batches, 
              steps_per_epoch= 4, 
              validation_data= val_batches, 
              validation_steps= 2, 
              epochs= epochs, 
              verbose= 1)

Epoch 1/10
4/4 [==============================] - 24s 7s/step - loss: 0.8186 - accuracy: 0.5938 - val_loss: 1.0791 - val_accuracy: 0.6719
Epoch 2/10
4/4 [==============================] - 23s 6s/step - loss: 0.6000 - accuracy: 0.7266 - val_loss: 0.9126 - val_accuracy: 0.7656
Epoch 3/10
4/4 [==============================] - 24s 7s/step - loss: 0.4960 - accuracy: 0.8047 - val_loss: 0.6560 - val_accuracy: 0.7656
Epoch 4/10
4/4 [==============================] - 25s 7s/step - loss: 0.5284 - accuracy: 0.7734 - val_loss: 0.6783 - val_accuracy: 0.7969
Epoch 5/10
4/4 [==============================] - 23s 7s/step - loss: 0.4276 - accuracy: 0.8208 - val_loss: 0.8826 - val_accuracy: 0.7656
Epoch 6/10
4/4 [==============================] - 21s 5s/step - loss: 0.3570 - accuracy: 0.8868 - val_loss: 0.4447 - val_accuracy: 0.8281
Epoch 7/10
4/4 [==============================] - 20s 6s/step - loss: 0.3169 - accuracy: 0.9057 - val_loss: 0.6670 - val_accuracy: 0.7500
Epoch 8/10
4/4 [==================

In [42]:
test_loss, test_acc= new_model.evaluate(test_batches)
print(f'Test accuracy = {test_acc: .4f} - test loss = {test_loss: 4f}')

15/15 [==============================] - 52s 3s/step - loss: 0.2695 - accuracy: 0.9246
Test accuracy =  0.9246 - test loss =  0.269512
